In [138]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from composio_openai import ComposioToolSet, Action
import re
from datetime import datetime, timedelta
import dateparser
import tzlocal
from dateutil import parser as dateutil_parser

In [139]:
def expand_time_phrases(prompt):
    match = re.search(r'from (\d{1,2}(?::\d{2})?\s*[ap]m)\s*to\s*(\d{1,2}(?::\d{2})?\s*[ap]m)', prompt, re.I)
    if not match:
        return prompt
    start_str, end_str = match.groups()
    local_tz = tzlocal.get_localzone()
    print("Detected local timezone:", local_tz)
    now = datetime.now(local_tz)
    today_base = now.replace(hour=0, minute=0, second=0, microsecond=0)
    start_dt = dateutil_parser.parse(start_str, default=today_base)
    end_dt = dateutil_parser.parse(end_str, default=today_base)
    if start_dt.tzinfo is None:
        start_dt = local_tz.localize(start_dt)
    if end_dt.tzinfo is None:
        end_dt = local_tz.localize(end_dt)
    print("DEBUG: start_dt", start_dt, "end_dt", end_dt)
    start_iso = start_dt.isoformat(timespec='minutes')
    end_iso = end_dt.isoformat(timespec='minutes')
    # Remove any "am"/"pm" after replacement to avoid parsing confusion
    new_prompt = re.sub(
        r'from (\d{1,2}(?::\d{2})?\s*[ap]m)\s*to\s*(\d{1,2}(?::\d{2})?\s*[ap]m)',
        f"from {start_iso} to {end_iso}",
        prompt
    )
    return new_prompt

In [140]:
load_dotenv() 

True

In [141]:
composio_key = os.environ.get("COMPOSIO_API_KEY")
openai_key = os.environ.get("OPENAI_API_KEY")

In [142]:
print("Composio key loaded:", bool(composio_key))
print("OpenAI key loaded:", bool(openai_key))

Composio key loaded: True
OpenAI key loaded: True


In [143]:
# Initialize OpenAI client
openai_client = OpenAI(api_key=openai_key)

In [144]:
# Initialize the Composio Tool Set
composio_tool_set = ComposioToolSet(api_key=composio_key)

In [145]:
tools = composio_tool_set.get_tools(
    actions=[Action.GITHUB_STAR_A_REPOSITORY_FOR_THE_AUTHENTICATED_USER,
             Action.GOOGLECALENDAR_CREATE_EVENT
    ]
)

In [146]:
user_prompt = ("Create a Google Calendar event called 'meeting: product direction and price discussion', "
"time: today from 4 pm to 6 pm, "
"description: 'showcase project progress and updates',"
"attendees: jay20020109@gmail.com, mh989@cornell.edu, z00183@mpinfo.com.tw."
)

In [147]:
expanded = expand_time_phrases(user_prompt)
print("DEBUG:", expanded)

Detected local timezone: Etc/UTC
DEBUG: start_dt 2025-06-12 16:00:00+00:00 end_dt 2025-06-12 18:00:00+00:00
DEBUG: Create a Google Calendar event called 'meeting: product direction and price discussion', time: today from 2025-06-12T16:00+00:00 to 2025-06-12T18:00+00:00, description: 'showcase project progress and updates',attendees: jay20020109@gmail.com, mh989@cornell.edu, z00183@mpinfo.com.tw.


In [148]:
#Expand the time phrases in the prompt
my_task = expand_time_phrases(user_prompt)

Detected local timezone: Etc/UTC
DEBUG: start_dt 2025-06-12 16:00:00+00:00 end_dt 2025-06-12 18:00:00+00:00


In [149]:
assistant_instruction = "You are a super intelligent personal assistant"
assistant = openai_client.beta.assistants.create(
    name="Personal Assistant",
    instructions=assistant_instruction,
    model="gpt-4-turbo",
    tools=tools,
)

In [150]:
thread = openai_client.beta.threads.create()

/tmp/ipykernel_5270/3829351525.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = openai_client.beta.threads.create()


In [151]:
message = openai_client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=my_task
)

/tmp/ipykernel_5270/4100934983.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = openai_client.beta.threads.messages.create(


In [152]:
run = openai_client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

/tmp/ipykernel_5270/2438797514.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = openai_client.beta.threads.runs.create(


In [153]:
response_after_tool_calls = composio_tool_set.wait_and_handle_assistant_tool_calls(
    client=openai_client,
    run=run,
    thread=thread,
)

In [154]:
print(response_after_tool_calls)

Run(id='run_oVTfTlvnasAIctL5Ot9fomKX', assistant_id='asst_Jydmy6PAKvRwy0rXCUgdOoeU', cancelled_at=None, completed_at=1749713356, created_at=1749713341, expires_at=None, failed_at=None, incomplete_details=None, instructions='You are a super intelligent personal assistant', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1749713352, status='completed', thread_id='thread_Kb78nIZ5vFpsHpI9PgWFPpqA', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='GITHUB_STAR_A_REPOSITORY_FOR_THE_AUTHENTICATED_USER', description='Stars an existing and accessible repository for the authenticated user; this action is idempotent and succeeds even if the repository is already starred.', parameters={'properties': {'owner': {'description': 'The username of the account that owns the repository. This field is not case-sensitive. Pleas

In [155]:
# Fetch the latest messages from the thread to get the assistant's summary
messages = openai_client.beta.threads.messages.list(thread_id=thread.id)

/tmp/ipykernel_5270/29257048.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = openai_client.beta.threads.messages.list(thread_id=thread.id)


In [156]:
# Usually, the last message will be the assistant's output after tool execution
if messages.data:
    last_message = messages.data[0]  # The newest message is usually at index 0
    print("\n=== Assistant's Action Summary ===")
    print(last_message.content[0].text.value)
else:
    print("No messages found in thread.")


=== Assistant's Action Summary ===
The event titled "meeting: product direction and price discussion" has been successfully created. It is scheduled for June 12, 2025, from 16:00 to 18:00 UTC. Here are the details:

- **Description:** showcase project progress and updates
- **Attendees:**
  - jay20020109@gmail.com (Organizer)
  - mh989@cornell.edu
  - z00183@mpinfo.com.tw

You can view and manage the event via [this link](https://www.google.com/calendar/event?eid=dDNndjNsMG9jajJzdXFjcWtldXBncWhvcGsgamF5MjAwMjAxMDlAbQ).

If you need to make any changes or add additional details, let me know!
